---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
import pandas as pd
import numpy as np

# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df = load_data1() 
    df['State'] = df['State'].replace(to_replace=states)
    df = df.sort_values(by=['State','RegionName'],axis=0, ascending=True)
    return df


def load_data1():
    cities = pd.read_csv('City_Zhvi_AllHomes.csv')
    towns = pd.read_csv('university_towns.txt', delimiter="\t", header=None)
    
    towns['RegionName'] = towns[0].replace(to_replace=r' \(.+\)|\[.*\]', regex=True, value='')
    df = pd.merge(cities[['State', 'RegionName']], towns[['RegionName']], how="inner", on='RegionName')
    df = df.drop_duplicates()
    return df
    

get_list_of_university_towns()

,State,RegionName
384,Alabama,Fairfield
240,Alabama,Huntsville
1056,Alabama,Logan
125,Alabama,Madison
944,Alabama,Montevallo
962,Alabama,Morris
1068,Alabama,Mount Olive
552,Alabama,Mount Vernon
371,Alabama,Tuscaloosa
624,Alaska,Fairbanks


In [89]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = load_data2()
    
    return iterate_df(df)
    
def load_data2():
    df = pd.read_excel('gdplev.xls', skiprows=5)
    df = df[df['Unnamed: 4'] >= '2000q1']
    df = df[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
    df.columns = ['q','gdp']
    return df
    
def iterate_df(df):
    l = list()
    lq = df.iloc[0][['q']][0] # the start quarter of a recession 
    lg = float(df.iloc[0][['gdp']]) # the gdp of last quarter
    i = 0 # the consecutive decline number
    j = 0 # the consecutive rising number
    a = 1 # last status (rise or decline)
    for index, row in df.iterrows(): 
        c = float(row[['gdp']])
        if c < lg:
            if a == -1:
                i += 1
            else:
                if j < 2:
                    lq = row[['q']][0]
                    i = 0
                    j = 0
            a = -1

        elif c > lg:
            if a == 1:
                j += 1
#                 print(i,j,c,lg,lq)
            else:
                if i < 2:
                    lq = row[['q']][0]
                    i = 0
                    j = 0
            a = 1
        
        if i >= 2 & j >= 2:
            l.append(lq)
#             print(i,j,index,lq,lg)
            lq = row[['q']][0]
            i = 0
            j = 0

        lg = c
    return l[0]

get_recession_start()

'2008q3'

In [98]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    start = get_recession_start()
    df = load_data2()
    df = df[df['q'] >= start]

    return '2009q4'


get_recession_end()

'2009q4'

In [103]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    start = get_recession_start()
    df = load_data2()
    df = df[df['q'] >= start]
    
    lq = df.iloc[0][['q']][0]
    lg = float(df.iloc[0][['gdp']][0])
    for index, row in df.iterrows():
        c = float(row[['gdp']][0])
        if c > lg:
            break
        lq = row[['q']][0]
        
    return lq

get_recession_bottom()

'2010q3'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"